# 02. 다층 퍼셉트론 (Multi-Layer Perceptron)

은닉층을 추가하여 비선형 문제를 해결하는 MLP를 구현합니다.

## 학습 목표
- 다층 신경망 구조 이해
- 역전파(Backpropagation) 알고리즘 이해
- 활성화 함수의 역할 이해
- MNIST 손글씨 분류

In [ ]:
import torch
import matplotlib.pyplot as plt
import numpy as np

torch.manual_seed(42)

## 1. XOR 문제

퍼셉트론의 한계를 보여주는 대표적인 예시입니다.

In [ ]:
# XOR 데이터
X_xor = torch.tensor([[0, 0], [0, 1], [1, 0], [1, 1]], dtype=torch.float32)
y_xor = torch.tensor([0, 1, 1, 0], dtype=torch.float32)

# 시각화
plt.figure(figsize=(6, 5))
plt.scatter(X_xor[y_xor == 0, 0], X_xor[y_xor == 0, 1], c='blue', s=200, label='Class 0')
plt.scatter(X_xor[y_xor == 1, 0], X_xor[y_xor == 1, 1], c='red', s=200, label='Class 1')
plt.xlabel('x1')
plt.ylabel('x2')
plt.title('XOR Problem - Not Linearly Separable')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

## 2. 다층 퍼셉트론(MLP) 구조

```
입력층 → 은닉층 → 출력층
  x    → h=σ(W₁x+b₁) → y=W₂h+b₂
```

은닉층의 비선형 활성화 함수가 핵심입니다!

In [ ]:
from mlfs.nn.layers import Linear
from mlfs.nn.activations import relu, sigmoid
from mlfs.nn.losses import BCELoss
from mlfs.nn.optim import Adam

In [ ]:
# XOR을 위한 간단한 MLP
class SimpleMLP:
    def __init__(self):
        self.fc1 = Linear(2, 4)  # 입력 → 은닉
        self.fc2 = Linear(4, 1)  # 은닉 → 출력
    
    def forward(self, x):
        h = relu(self.fc1(x))    # 은닉층 + ReLU
        out = sigmoid(self.fc2(h))  # 출력층 + Sigmoid
        return out.squeeze()
    
    def parameters(self):
        return self.fc1.parameters() + self.fc2.parameters()
    
    def __call__(self, x):
        return self.forward(x)

In [ ]:
# XOR 학습
model = SimpleMLP()
optimizer = Adam(model.parameters(), lr=0.1)
criterion = BCELoss()

losses = []
for epoch in range(1000):
    output = model(X_xor)
    loss = criterion(output, y_xor)
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    losses.append(loss.item())
    
    if (epoch + 1) % 200 == 0:
        predictions = (output > 0.5).float()
        accuracy = (predictions == y_xor).float().mean()
        print(f"Epoch {epoch + 1}: Loss = {loss.item():.4f}, Accuracy = {accuracy:.4f}")

In [ ]:
# 결과 확인
with torch.no_grad():
    output = model(X_xor)
    predictions = (output > 0.5).float()

print("\nXOR Results:")
for i in range(4):
    print(f"Input: {X_xor[i].tolist()}, Predicted: {predictions[i].item():.0f}, Actual: {y_xor[i].item():.0f}")

## 3. MNIST 분류

이제 실제 데이터셋인 MNIST를 분류해봅니다.

In [ ]:
from mlfs.utils.data import load_mnist
from mlfs.utils.viz import plot_images

# 데이터 로드
X_train, y_train = load_mnist(train=True, flatten=True)
X_test, y_test = load_mnist(train=False, flatten=True)

print(f"Train: {X_train.shape}, Test: {X_test.shape}")

In [ ]:
# 샘플 이미지 확인
X_sample, y_sample = load_mnist(train=True, flatten=False)
plot_images(X_sample[:10], labels=y_sample[:10], n_rows=2, n_cols=5)

In [ ]:
from mlfs.nn.models import MLP
from mlfs.nn.losses import CrossEntropyLoss

# MLP 모델 생성: 784 → 256 → 128 → 10
model = MLP([784, 256, 128, 10])
optimizer = Adam(model.parameters(), lr=0.001)
criterion = CrossEntropyLoss()

print(model)

In [ ]:
# 학습 설정
epochs = 20
batch_size = 128

train_losses = []
train_accs = []
test_accs = []

n_train = len(X_train)

for epoch in range(epochs):
    model.train()
    epoch_loss = 0
    correct = 0
    
    # 셔플
    indices = torch.randperm(n_train)
    
    for i in range(0, n_train, batch_size):
        batch_idx = indices[i:i+batch_size]
        X_batch = X_train[batch_idx]
        y_batch = y_train[batch_idx]
        
        # 순전파
        logits = model(X_batch)
        loss = criterion(logits, y_batch)
        
        # 역전파
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        epoch_loss += loss.item() * len(batch_idx)
        correct += (logits.argmax(dim=1) == y_batch).sum().item()
    
    train_loss = epoch_loss / n_train
    train_acc = correct / n_train
    train_losses.append(train_loss)
    train_accs.append(train_acc)
    
    # 테스트 정확도
    model.eval()
    with torch.no_grad():
        test_logits = model(X_test)
        test_acc = (test_logits.argmax(dim=1) == y_test).float().mean().item()
    test_accs.append(test_acc)
    
    print(f"Epoch {epoch + 1:2d}: Loss = {train_loss:.4f}, Train Acc = {train_acc:.4f}, Test Acc = {test_acc:.4f}")

In [ ]:
# 결과 시각화
fig, axes = plt.subplots(1, 2, figsize=(12, 4))

axes[0].plot(train_losses)
axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('Loss')
axes[0].set_title('Training Loss')
axes[0].grid(True, alpha=0.3)

axes[1].plot(train_accs, label='Train')
axes[1].plot(test_accs, label='Test')
axes[1].set_xlabel('Epoch')
axes[1].set_ylabel('Accuracy')
axes[1].set_title('Accuracy')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"\nFinal Test Accuracy: {test_accs[-1]:.4f}")

In [ ]:
# 예측 결과 시각화
model.eval()
with torch.no_grad():
    predictions = model.predict(X_test[:20])

X_test_img, _ = load_mnist(train=False, flatten=False)
plot_images(X_test_img[:20], labels=y_test[:20], predictions=predictions, n_rows=4, n_cols=5)

## 요약

이 노트북에서 배운 내용:

1. **MLP 구조**: 여러 층을 쌓아 비선형 문제 해결
2. **활성화 함수**: ReLU, Sigmoid 등으로 비선형성 추가
3. **역전파**: 체인룰을 사용한 그래디언트 계산
4. **MNIST 분류**: ~97% 정확도 달성

다음 노트북에서는 이미지에 더 적합한 **CNN**을 학습합니다.